In [7]:
import pandas as pd
import random

# 항목 정의
genders = ["남성", "여성"]
foods = ["참치", "김치찌개", "햄버거", "비빔밥", "샐러드", "삼겹살", "라면", "스시", "과일", "샌드위치"]
improved_lifestyles = [
    "건강 중시", "가성비 중시", "간편함 추구", "프리미엄 선호",
    "친환경 생활", "가족 중심", "여가 중시", "디지털 중심", "브랜드 충성"
]
usage_freq = ["주 3회 이상", "주 1~2회", "월 2~3회", "월 1회 미만", "거의 안함"]

# 나이대별 인구비율
age_distribution = {
    "20대": 0.17,
    "30대": 0.18,
    "40대": 0.20,
    "50대": 0.18,
    "60대": 0.14,
    "70대": 0.09,
    "80대 이상": 0.04
}

# 나이대별 가족구성 매핑 (현실적)
age_family_map = {
    "20대": ["1인 가구", "2인 부부", "부모동거", "자녀 1명"],
    "30대": ["2인 부부", "자녀 1명", "자녀 2명", "부모동거"],
    "40대": ["자녀 1명", "자녀 2명", "자녀 3명이상", "1인 가구", "2인 부부"],
    "50대": ["자녀 1명", "자녀 2명", "자녀 3명이상", "1인 가구"],
    "60대": ["자녀 1명", "자녀 2명", "자녀 3명이상", "1인 가구", "2인 부부"],
    "70대": ["1인 가구", "2인 부부"],
    "80대 이상": ["1인 가구", "2인 부부"]
}

# 나이대별 직업군 매핑 (현실적)
age_job_map = {
    "20대": ["학생", "회사원", "프리랜서", "전업주부"],
    "30대": ["회사원", "프리랜서", "자영업", "공무원", "전업주부"],
    "40대": ["회사원", "자영업", "공무원", "프리랜서", "전업주부"],
    "50대": ["회사원", "자영업", "공무원", "프리랜서", "전업주부"],
    "60대": ["자영업", "공무원", "은퇴", "전업주부"],
    "70대": ["자영업", "은퇴"],
    "80대 이상": ["은퇴"]
}

# 총 응답 수 설정
total_rows = 300
age_counts = {age: round(total_rows * pct) for age, pct in age_distribution.items()}

# 총합 조정
while sum(age_counts.values()) != total_rows:
    diff = total_rows - sum(age_counts.values())
    max_group = max(age_counts, key=age_counts.get)
    age_counts[max_group] += diff

# 조건 적용하여 직업 선택 함수 정의
def choose_job(age, gender, family):
    possible_jobs = age_job_map[age].copy()
    
    # 전업주부 제한 조건
    if "전업주부" in possible_jobs:
        if gender == "남성" or (gender == "여성" and family == "1인 가구"):
            possible_jobs.remove("전업주부")

    return random.choice(possible_jobs) if possible_jobs else "기타"

# 데이터 생성
data = []
row_id = 1

for age, count in age_counts.items():
    for _ in range(count):
        gender = random.choice(genders)
        family = random.choice(age_family_map[age])
        job = choose_job(age, gender, family)

        row = {
            "응답자ID": f"{row_id:03}",
            "성별": gender,
            "나이대": age,
            "가족구성": family,
            "직업": job,
            "좋아하는음식": ", ".join(random.sample(foods, k=random.randint(1, 3))),
            "라이프스타일": ", ".join(random.sample(improved_lifestyles, k=random.randint(1, 2))),
            "동원제품이용횟수": random.choice(usage_freq)
        }
        data.append(row)
        row_id += 1

# 저장
df = pd.DataFrame(data)
df.to_csv("페르소나_test.csv", index=False, encoding="utf-8-sig")

In [1]:
import json

# 항목 정의
persona_categories = {
    "나이": ["20대", "30대", "40대", "50대", "60대이상"],
    "성별": ["남", "여"],
    "가족구성": ["1인가구", "부모동거", "부부", "자녀1명", "자녀2명이상"],
    "고객취향": [
        "통조림/즉석/면류", "생수/음료/커피", "과자/떡/베이커리",
        "냉장/냉동/간편식", "유제품", "건강식품"
    ],
    "고객가치(RFM)": ["VIP", "우수고객", "잠재우수고객", "신규고객", "잠재이탈고객", "이탈/휴면고객"],
    "라이프스타일": ["트랜드추종", "가격민감", "브랜드선호", "건강중시"]
}

# 사용자 선택 함수
def get_user_input(category, options):
    print(f"\n{category} 선택:")
    for i, option in enumerate(options, 1):
        print(f"  {i}. {option}")
    while True:
        choice = input(f"번호를 입력하세요 (1~{len(options)}): ").strip()
        if choice.isdigit() and 1 <= int(choice) <= len(options):
            return options[int(choice) - 1]
        print("유효한 번호를 입력해주세요.")
        
# 고객취향 다중 선택 함수 (최대 3개)
def get_multiple_choices(category, options, max_select=3):
    print(f"\n{category} 선택 (최대 {max_select}개까지 선택, 쉼표로 구분):")
    for i, option in enumerate(options, 1):
        print(f"  {i}. {option}")
    while True:
        choice_str = input(f"번호 입력 (예: 1,3,6): ").strip()
        indices = [s.strip() for s in choice_str.split(",")]
        if all(s.isdigit() and 1 <= int(s) <= len(options) for s in indices):
            selected = list({options[int(i)-1] for i in indices})
            if 1 <= len(selected) <= max_select:
                return selected
        print("올바른 번호 형식 또는 개수를 입력해주세요.")

# 페르소나 구성
def build_persona():
    persona = {}
    for category, options in persona_categories.items():
        if category == "고객취향":
            persona[category] = get_multiple_choices(category, options, max_select=3)
        else:
            persona[category] = get_user_input(category, options)
    return persona

# 자연어 프롬프트 생성
def format_persona_prompt(p):
    return (
        f"당신은 {p['나이']} {p['성별']}이며, {p['가족구성']}입니다. "
        f"{p['고객가치(RFM)']}이며, "
        f"{' / '.join(p['고객취향'])}을(를) 선호하고, "
        f"{p['라이프스타일']}한 라이프스타일을 가지고 있습니다."
    )

# 실행
print("페르소나를 선택해 주세요!")
persona = build_persona()
query = format_persona_prompt(persona)

print("\n생성된 쿼리:")
print(query)

페르소나를 선택해 주세요!

나이 선택:
  1. 20대
  2. 30대
  3. 40대
  4. 50대
  5. 60대이상
번호를 입력하세요 (1~5): 1

성별 선택:
  1. 남
  2. 여
번호를 입력하세요 (1~2): 2

가족구성 선택:
  1. 1인가구
  2. 부모동거
  3. 부부
  4. 자녀1명
  5. 자녀2명이상
번호를 입력하세요 (1~5): 2

고객취향 선택 (최대 3개까지 선택, 쉼표로 구분):
  1. 통조림/즉석/면류
  2. 생수/음료/커피
  3. 과자/떡/베이커리
  4. 냉장/냉동/간편식
  5. 유제품
  6. 건강식품
번호 입력 (예: 1,3,6): 1,2,4

고객가치(RFM) 선택:
  1. VIP
  2. 우수고객
  3. 잠재우수고객
  4. 신규고객
  5. 잠재이탈고객
  6. 이탈/휴면고객
번호를 입력하세요 (1~6): 4

라이프스타일 선택:
  1. 트랜드추종
  2. 가격민감
  3. 브랜드선호
  4. 건강중시
번호를 입력하세요 (1~4): 1

생성된 쿼리:
당신은 20대 여이며, 부모동거입니다. 신규고객이며, 생수/음료/커피 / 통조림/즉석/면류 / 냉장/냉동/간편식을(를) 선호하고, 트랜드추종한 라이프스타일을 가지고 있습니다.


In [2]:
print(query)

당신은 20대 여이며, 부모동거입니다. 신규고객이며, 생수/음료/커피 / 통조림/즉석/면류 / 냉장/냉동/간편식을(를) 선호하고, 트랜드추종한 라이프스타일을 가지고 있습니다.


In [3]:
import pandas as pd
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain_upstage import UpstageEmbeddings
from dotenv import load_dotenv
import os

# CSV 파일 로드
df = pd.read_csv("페르소나_test.csv")

# 자연어 문장 변환 함수
def row_to_sentence(row):
    return (
        f"이 사람은 {row['나이대']} {row['성별']}이며, 가족구성은 {row['가족구성']}입니다. "
        f"{row['직업']}으로 일하고 있으며, 좋아하는 음식은 {row['좋아하는음식']}입니다. "
        f"라이프스타일은 {row['라이프스타일']}이며, 동원 제품은 {row['동원제품이용횟수']} 정도 이용합니다."
    )

# 문서 리스트 생성
documents = df.apply(lambda row: Document(page_content=row_to_sentence(row)), axis=1).tolist()

# Upstage 임베딩 모델 사용
load_dotenv()
embedding = UpstageEmbeddings(
#     model = "solar-embedding-1-large"
    model = "embedding-query"
)

# Chroma 벡터DB 생성
persist_dir = "chroma_upstage"
if os.path.exists(persist_dir):
    import shutil
    shutil.rmtree(persist_dir)

vectorstore = Chroma.from_documents(documents, embedding=embedding, persist_directory=persist_dir)
vectorstore.persist()

# 사용자 쿼리 입력
query = query

# 유사 페르소나 검색
results_with_score = vectorstore.similarity_search_with_score(query, k=3)

print("\n유사한 페르소나 (유사도 포함):")
for i, (doc, score) in enumerate(results_with_score, 1):
    similarity = 1 - score
    print(f"\n[Top {i}] (유사도: {similarity:.4f})")
    print(doc.page_content)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13732\536792118.py:36: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()



유사한 페르소나 (유사도 포함):

[Top 1] (유사도: 0.9425)
이 사람은 20대 여성이며, 가족구성은 부모동거입니다. 회사원으로 일하고 있으며, 좋아하는 음식은 라면입니다. 라이프스타일은 여가 중시, 브랜드 충성이며, 동원 제품은 월 1회 미만 정도 이용합니다.

[Top 2] (유사도: 0.9558)
이 사람은 30대 여성이며, 가족구성은 부모동거입니다. 공무원으로 일하고 있으며, 좋아하는 음식은 라면, 햄버거입니다. 라이프스타일은 친환경 생활, 간편함 추구이며, 동원 제품은 월 1회 미만 정도 이용합니다.

[Top 3] (유사도: 0.9686)
이 사람은 30대 남성이며, 가족구성은 부모동거입니다. 회사원으로 일하고 있으며, 좋아하는 음식은 샌드위치, 샐러드, 라면입니다. 라이프스타일은 간편함 추구, 여가 중시이며, 동원 제품은 거의 안함 정도 이용합니다.
